In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jocelyndumlao/shoppingappreviews-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/shoppingappreviews-dataset


In [2]:
!kaggle datasets download jocelyndumlao/shoppingappreviews-dataset

Dataset URL: https://www.kaggle.com/datasets/jocelyndumlao/shoppingappreviews-dataset
License(s): CC0-1.0
 98%|███████████████████████████████████████▏| 139M/142M [00:02<00:00, 47.8MB/s]
100%|████████████████████████████████████████| 142M/142M [00:02<00:00, 71.4MB/s]


In [3]:
!unzip shoppingappreviews-dataset.zip

Archive:  shoppingappreviews-dataset.zip
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Alibaba.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Aliexpress.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Amazon shopping.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Daraz Online Shopping App.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Flipkart.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Lazada.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Meesho.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Myntra.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Shein.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Snapdeal.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Walmart.csv  
  inflating: ShoppingAppReviews Dataset/ShoppingAppReviews/json/Alibaba.json  
  infla

# **Important Libraries**

In [4]:
# Important Libraries
import string
import numpy as np
import re
import pickle
import unicodedata
import html
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense, Dropout,GRU,Bidirectional
import subprocess
import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('omw-1.4')
# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package wordnet to /kaggle/working/...


# **Load Dataset**

In [5]:
# Read the file 
df=pd.read_csv('ShoppingAppReviews Dataset/ShoppingAppReviews/csv/Aliexpress.csv')
df.head()

,reviewId,content,score,thumbsUpCount,at,replyContent,repliedAt,appName
0,d0ea4cf3-8afc-4a6e-b154-d55236e307a2,Made a brand new account and was immediately k...,1,1,1723858282000,NaN,NaN,Aliexpress
1,975c7ce2-0263-46ab-8f41-2f3e7b26de26,"Great prices, mostly good products (mistakes h...",5,134,1723073042000,NaN,NaN,Aliexpress
2,e471d59f-046e-4f2a-968f-67d9775ebddd,Why is it that each update the search gets wor...,1,5,1723626891000,NaN,NaN,Aliexpress
3,841ffdf3-27f0-4733-85bc-c517b876eee6,Shopping on here in recent years has been a pl...,3,72,1722019932000,Thank you for your feedback. We're always tryi...,1.723133e+12,Aliexpress
4,065f1388-b323-4bbb-a1f1-029d262b56b3,Love AliExpress app and online! One of my favo...,5,160,1722911755000,NaN,NaN,Aliexpress


# **EDA**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126000 entries, 0 to 125999
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   reviewId       126000 non-null  object 
 1   content        125999 non-null  object 
 2   score          126000 non-null  int64  
 3   thumbsUpCount  126000 non-null  int64  
 4   at             126000 non-null  int64  
 5   replyContent   1436 non-null    object 
 6   repliedAt      1436 non-null    float64
 7   appName        126000 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 7.7+ MB


In [7]:
# Chech for Duplicated Values
df.duplicated().sum()

0

# **Spliting and Preprocessing**

In [8]:
X=df['content']
y=df['score']

In [9]:
X.head()

0    Made a brand new account and was immediately k...
1    Great prices, mostly good products (mistakes h...
2    Why is it that each update the search gets wor...
3    Shopping on here in recent years has been a pl...
4    Love AliExpress app and online! One of my favo...
Name: content, dtype: object

In [10]:
y.unique()

array([1, 5, 3, 2, 4])

In [11]:
def label(score):
    if score<3:
        return 0
    elif score == 3:
        return 1
    else:
        return 2
y=y.apply(label)
y.unique()

array([0, 2, 1])

In [12]:
y.value_counts() # target is imbalanced 

score
2    88968
0    30667
1     6365
Name: count, dtype: int64

In [15]:
def normalize_text(text):
    """
    Normalize text for sentiment analysis by retaining sentiment-bearing features like emojis,
    negations, and tone indicators while cleaning unnecessary elements.
    :param text: Input text to be normalized
    :return: Normalized text as a string
    """
    # Initialize lemmatizer and tokenizer
    lemmatizer = WordNetLemmatizer()
    tokenizer = TweetTokenizer()

    # Custom stopwords list (excluding negations and sentiment-bearing words)
    stop_words = set(stopwords.words('english')) - {'not', 'no', 'never', 'nor', 'none', 'nothing', 'neither'}

    # Replace HTML entities and special characters
    text = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'") \
        .replace('nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'") \
        .replace('<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.') \
        .replace(' @-@ ', '-').replace('\\', ' \\ ')
    text = re.sub(r'  +', ' ', html.unescape(text))

    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # Remove emails
    text = re.sub(r'\S+@\S+\.\S+', '', text)

    # Map emojis to their textual equivalents
    text = emoji.demojize(text, delimiters=(" ", " "))  # Convert emojis to text

    # Remove non-ASCII characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Tokenize the text
    words = tokenizer.tokenize(text)

    # Remove stopwords (excluding negations)
    words = [word for word in words if word not in stop_words]

    # Lemmatize words (preserve sentiment-bearing words)
    words = [lemmatizer.lemmatize(word, pos='v') for word in words]

    # Retain exclamation marks and question marks for tone
    words = [word for word in words if word not in string.punctuation or word in {'!', '?'}]

    return ' '.join(words)

In [16]:
X = X.astype(str).apply(normalize_text)
X.head()

0    make brand new account immediately kick app go...
1    great price mostly good products mistake happe...
2    update search get worse app can't decide app w...
3    shop recent years pleasure items always arrive...
4    love aliexpress app online ! one favorite apps...
Name: content, dtype: object

In [17]:
# Making Training and Testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((100800,), (25200,), (100800,), (25200,))

In [18]:
# Prepare tokenizer
t = Tokenizer(num_words=10000)
t.fit_on_texts(X_train)
# Calculate vocabulary size
vocab_size = len(t.word_index) + 1
max_len=100
# Integer encode
encoded_train_docs = t.texts_to_sequences(X_train)
# Integer encode
encoded_test_docs = t.texts_to_sequences(X_test)

# Pad documents to a max length of 100 words
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_len, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_len, padding='post')

# **Word2Vec (CBOW)**

In [19]:
tokenized_padded_sentences = [list(map(str, sentence)) for sentence in padded_train_docs]

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_padded_sentences, vector_size=50, window=5, min_count=1, workers=4)

#Create embedding matrix
vocab = word2vec_model.wv.key_to_index
vocab_size=len(vocab) + 1
embedding_matrix = np.zeros((vocab_size, 50))

# Fill the embedding matrix with Word2Vec embeddings
for word, i in vocab.items():
    embedding_matrix[i] = word2vec_model.wv[word]

In [20]:
# Oversample the padded training data
ros = RandomOverSampler(random_state=42)
padded_train_docs, y_train = ros.fit_resample(padded_train_docs, y_train)

# **GRU Model**

In [21]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50,weights=[embedding_matrix],input_length=max_len, trainable=False),
    Bidirectional(GRU(256, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(GRU(256)),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │         500,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 500,050 (1.91 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 500,050 (1.91 MB)

In [22]:
model.fit(padded_train_docs, y_train, epochs=5,verbose=1,validation_split=0.2)

Epoch 1/5
5339/5339 ━━━━━━━━━━━━━━━━━━━━ 198s 36ms/step - accuracy: 0.7630 - loss: 0.5867 - val_accuracy: 0.4925 - val_loss: 1.1505
Epoch 2/5
5339/5339 ━━━━━━━━━━━━━━━━━━━━ 190s 36ms/step - accuracy: 0.8449 - loss: 0.4122 - val_accuracy: 0.8385 - val_loss: 0.5267
Epoch 3/5
5339/5339 ━━━━━━━━━━━━━━━━━━━━ 190s 36ms/step - accuracy: 0.9054 - loss: 0.2755 - val_accuracy: 0.8475 - val_loss: 0.4798
Epoch 4/5
5339/5339 ━━━━━━━━━━━━━━━━━━━━ 190s 36ms/step - accuracy: 0.9255 - loss: 0.2218 - val_accuracy: 0.8315 - val_loss: 0.5222
Epoch 5/5
5339/5339 ━━━━━━━━━━━━━━━━━━━━ 190s 36ms/step - accuracy: 0.9256 - loss: 0.2237 - val_accuracy: 0.9123 - val_loss: 0.2804


In [23]:
# Evaluate model
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print(f'Test Accuracy: {(accuracy)}, Loss: {loss}')

Test Accuracy: 0.8538492321968079, Loss: 0.5312950015068054


# **Saving the Model**

In [24]:
# Save the model and tokenizer in the working directory
model.save('/kaggle/working/sentiment_model.h5')
with open('/kaggle/working/tokenizer.pkl', 'wb') as file:
    pickle.dump(t, file)